In [ ]:
from pathlib import Path

# Automatically get the base path of your project
base_path = Path.cwd().parents[0]  # adjust .parents[0] if needed
print("Base path of the project:", base_path)

In [ ]:
# Paste the file path of the database you want to compare with the metagenomics database between the quotes
query_db = r""

In [ ]:
import os
import subprocess
from pathlib import Path

# === Configuration ===
base_path = Path.cwd().parents[0]
diamond_exe = base_path / "software_tools" / "diamond.exe"
diamond_db_path = os.path.splitext(query_db)[0]  # same path, .dmnd suffix will be added

# === Check and create DIAMOND DB ===
def ensure_diamond_db(fasta_file: str, diamond_db: str):
    if not os.path.exists(diamond_db + ".dmnd"):
        print(f"Creating DIAMOND database for: {fasta_file}")
        subprocess.run([
            diamond_exe, "makedb",
            "--in", fasta_file,
            "-d", diamond_db
        ], check=True)
        print("DIAMOND DB created.")
    else:
        print(f"DIAMOND DB already exists: {diamond_db}.dmnd")

# === Run check/create for genus database ===
ensure_diamond_db(query_db, diamond_db_path)

In [ ]:
import subprocess
import os
from pathlib import Path
from tqdm import tqdm

# === Paths ===
base_path = Path.cwd().parents[0]
diamond_exe = base_path / "software_tools" / "diamond.exe"
metagen_db = base_path / "db_psm_results" / "Metagenomics_db" / "GW_proteins_CD_clean.fasta"
output_file = base_path / "db_results_analysis" / "Diamond_alignments" / "querydb_aligned_with_metagenomicsdb.m8"

# === Step 1: Estimate query size ===
def count_fasta_headers(fasta_path):
    with open(fasta_path) as f:
        return sum(1 for line in f if line.startswith(">"))

n_proteins = count_fasta_headers(metagen_db)
print(f"Estimating {n_proteins} proteins for alignment...")

# === Step 2: Run DIAMOND alignment with progress bar ===
print("Running DIAMOND alignment...")

with tqdm(total=n_proteins, desc="Aligning proteins", unit="seq") as pbar:
    process = subprocess.Popen([
        str(diamond_exe), "blastp",
        "--query", str(metagen_db),
        "--db", str(diamond_db_path) + ".dmnd",
        "--out", str(output_file),
        "--outfmt", "6",  # BLAST tabular format (.m8)
        "--threads", "4",
        "--max-target-seqs", "1",
        "--fast"
    ], stderr=subprocess.PIPE, text=True)

    # Update progress bar from stderr (if possible)
    while True:
        line = process.stderr.readline()
        if not line:
            break
        if "Processed" in line:
            try:
                processed = int(line.strip().split()[1])
                pbar.n = processed
                pbar.refresh()
            except:
                pass

    process.wait()
    pbar.n = n_proteins
    pbar.refresh()

print(f"Alignment complete. Output saved to: {output_file}")


In [ ]:
from pathlib import Path

# === Paths ===
alignment_result = output_file  # Output from the DIAMOND alignment

# === Load protein IDs from FASTA files ===
def load_fasta_ids(fasta_path):
    with open(fasta_path) as f:
        return {line[1:].strip().split()[0] for line in f if line.startswith(">")}

querydb_ids = load_fasta_ids(query_db)
meta_ids = load_fasta_ids(metagen_db)

# === Load aligned pairs (query ID -> subject ID) ===
querydb_ids = set()
aligned_meta_ids = set()

with open(alignment_result) as f:
    for line in f:
        query_id, subject_id = line.strip().split()[:2]
        aligned_meta_ids.add(query_id)
        querydb_ids.add(subject_id)

# === Compute set differences ===
querydb_found_in_meta = querydb_ids
querydb_missing_in_meta = querydb_ids - querydb_ids

print(f"✅ Total proteins in Diamond hits DB: {len(querydb_ids)}")
print(f"🔍 Diamond hits proteins found in metagenomics DB: {len(querydb_found_in_meta)}")
print(f"❌ Diamond hits proteins NOT found in metagenomics DB: {len(querydb_missing_in_meta)}")

print(f"\n✅ Total proteins in metagenomics DB: {len(meta_ids)}")
print(f"🔍 Metagenomics proteins that match Diamond hits DB: {len(aligned_meta_ids)}")
print(f"❌ Metagenomics proteins NOT matching Diamond hits DB: {len(meta_ids - aligned_meta_ids)}")
